In [493]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn import preprocessing


In [494]:
df = pd.read_pickle('data\otomoto_data.pickle')
df.head()

,price,Oferta od,Kategoria,Marka pojazdu,Model pojazdu,Rok produkcji,Przebieg,Pojemność skokowa,Rodzaj paliwa,Moc,...,Okres gwarancji producenta,Kierownica po prawej (Anglik),Generacja,VIN,Zarejestrowany jako zabytek,Homologacja ciężarowa,Średnie zużycie,Pojemność baterii,Autonomia,Czas ładowania
0,39 900 PLN,Firmy,Osobowe,Abarth,Inny,2009,158 000 km,1 368 cm3,Benzyna,215 KM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,72 900 PLN,Firmy,Osobowe,Abarth,500,2019,66 500 km,1 368 cm3,Benzyna,145 KM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,39 900 PLN,Osoby prywatnej,Osobowe,Abarth,500,2012,193 631 km,1 400 cm3,Benzyna,135 KM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,43 900 PLN,Osoby prywatnej,Osobowe,Abarth,500,2015,139 000 km,1 368 cm3,Benzyna,160 KM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,36 500 PLN,Osoby prywatnej,Osobowe,Abarth,500,2013,103 000 km,1 368 cm3,Benzyna,160 KM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [495]:
df_base = df.copy()

Initian adjustment of dtypes

In [496]:
df['price'].str.contains('EUR')

0        False
2        False
3        False
4        False
5        False
         ...  
22228    False
22229    False
22230    False
22231    False
22232    False
Name: price, Length: 130804, dtype: bool

In [497]:
# df[df['price'].str.contains('EUR')]['price'] = df[df['price'].str.contains('EUR')]['price'].str.replace(' EUR', '').str.replace(' ','').str.replace(',','.').astype('float')
# *4.61
# df.loc[df['price'].str.contains('EUR'), 'price'] 

df['price'] = np.where(df['price'].str.contains('EUR')==True, df['price'].str.replace(' EUR', '').str.replace(' ','').str.replace(',','.'), df['price'])

In [498]:
df.loc[~df['price'].str.contains('PLN'), 'price'] = df.loc[~df['price'].str.contains('PLN'), 'price'].astype('float').apply(lambda x: x*4.61).astype('str')


In [499]:
df.loc[~df['price'].str.contains('PLN'), 'price']

156                53780.26
157       55140.21000000001
206                48428.05
207      43260.240000000005
1169                85285.0
                ...        
19286    19357.390000000003
19968    27660.000000000004
21342               35958.0
21986    119399.00000000001
22012     57625.00000000001
Name: price, Length: 270, dtype: object

In [500]:
df['price'] = df['price'].str.replace(' PLN', '').str.replace(' ','').str.replace(',','.').astype('float')
df['Przebieg'] = df['Przebieg'].str.replace(' km', '').str.replace(' ','').astype('float')
df['Pojemność skokowa'] = df['Pojemność skokowa'].str.replace(' cm3', '').str.replace(' ','').astype('float')
df['Moc'] = df['Moc'].str.replace(' KM', '').str.replace(' ','').astype('float')
df['Liczba drzwi'] = df['Liczba drzwi'].str.replace(' KM', '').str.replace(' ','').astype('float')
df['Liczba miejsc'] = df['Liczba miejsc'].str.replace(' KM', '').str.replace(' ','').astype('float')


check which columns to fill, which to drop in df

In [501]:
len_df = df.shape[0]
fill_col_list = []
drop_col_list = []

for col in df.columns:
    num_na = df[col].isna().sum()
    if num_na == 0:
        continue
    elif num_na < len_df*0.2:
        fill_col_list.append(col)
    else:
        drop_col_list.append(col)
        
# new_df
print(fill_col_list)
print(drop_col_list)

['Przebieg', 'Pojemność skokowa', 'Moc', 'Skrzynia biegów', 'Napęd', 'Liczba drzwi', 'Liczba miejsc']
['Rodzaj koloru', 'VAT marża', 'Kraj pochodzenia', 'Pierwsza rejestracja', 'Zarejestrowany w Polsce', 'Bezwypadkowy', 'Serwisowany w ASO', 'Numer rejestracyjny pojazdu', 'Spalanie W Mieście', 'Faktura VAT', 'Wersja', 'Możliwość finansowania', 'Pierwszy właściciel', 'Tuning', 'Leasing', 'Opłata początkowa', 'Miesięczna rata', 'Liczba pozostałych rat', 'Wartość wykupu', 'Uszkodzony', 'Emisja CO2', 'Gwarancja dealerska (w cenie)', 'Filtr cząstek stałych', 'lub do (przebieg km)', 'Spalanie W Cyklu Mieszanym', 'Spalanie Poza Miastem', 'Rodzaj własności baterii', 'Okres gwarancji producenta', 'Kierownica po prawej (Anglik)', 'Generacja', 'VIN', 'Zarejestrowany jako zabytek', 'Homologacja ciężarowa', 'Średnie zużycie', 'Pojemność baterii', 'Autonomia', 'Czas ładowania']


In [502]:
df = df.drop(drop_col_list, axis = 1)
df

,price,Oferta od,Kategoria,Marka pojazdu,Model pojazdu,Rok produkcji,Przebieg,Pojemność skokowa,Rodzaj paliwa,Moc,Skrzynia biegów,Napęd,Typ nadwozia,Liczba drzwi,Liczba miejsc,Kolor,Stan,equipment,description
0,39900.0,Firmy,Osobowe,Abarth,Inny,2009,158000.0,1368.0,Benzyna,215.0,Manualna,Na przednie koła,Auta miejskie,3.0,5.0,Biały,Używane,"Radio,Klimatyzacja manualna,Elektryczne szyby ...",\n Sprzedam Abarth Grande Punto z 2009 ...
2,72900.0,Firmy,Osobowe,Abarth,500,2019,66500.0,1368.0,Benzyna,145.0,Manualna,Na przednie koła,Auta małe,3.0,4.0,Biały,Używane,"Radio,System nawigacji satelitarnej,Klimatyzac...",\n Samochody Używane Nowy Targ\n Ul. ...
3,39900.0,Osoby prywatnej,Osobowe,Abarth,500,2012,193631.0,1400.0,Benzyna,135.0,Manualna,Na przednie koła,Auta małe,3.0,4.0,Biały,Używane,"Apple CarPlay,Android Auto,Interfejs Bluetooth...",\n Sprzedam abartha 500 Fiat 1.4 T 135k...
4,43900.0,Osoby prywatnej,Osobowe,Abarth,500,2015,139000.0,1368.0,Benzyna,160.0,Manualna,Na przednie koła,Auta małe,3.0,4.0,Czerwony,Używane,,\n Fiat ABARTH - 1.4 Turbo - 160KM\n\nW...
5,36500.0,Osoby prywatnej,Osobowe,Abarth,500,2013,103000.0,1368.0,Benzyna,160.0,Manualna,Na przednie koła,Auta małe,3.0,4.0,Czarny,Używane,"Interfejs Bluetooth,Radio,Gniazdo USB,Klimatyz...",\n Witam.\nSprzedam Abarth 500\nRok pro...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22228,124900.0,Firmy,Osobowe,Peugeot,3008,2021,14000.0,1499.0,Diesel,130.0,Manualna,Na przednie koła,SUV,5.0,5.0,Niebieski,Używane,"Apple CarPlay,Android Auto,Interfejs Bluetooth...",\n Witam mam do zaoferowania ślicznego ...
22229,129900.0,Firmy,Osobowe,Peugeot,3008,2021,1.0,1199.0,Benzyna,130.0,Manualna,Na przednie koła,SUV,5.0,5.0,Szary,Nowe,"Interfejs Bluetooth,Radio,Gniazdo USB,System n...",\n ULIARCZYK SP. Z O.O.\r\nAutoryzowany...
22230,138300.0,Firmy,Osobowe,Peugeot,3008,2021,1.0,1199.0,Benzyna,130.0,Automatyczna,Na przednie koła,SUV,5.0,5.0,Biały,Nowe,"Interfejs Bluetooth,Radio,Gniazdo USB,System n...",\n ULIARCZYK SP. Z O.O.\r\nAutoryzowany...
22231,179000.0,Firmy,Osobowe,Peugeot,3008,2022,2.0,1199.0,Benzyna,130.0,Automatyczna,Na przednie koła,SUV,5.0,5.0,Szary,Nowe,"Interfejs Bluetooth,Radio,Gniazdo USB,System n...",\n M. i R. Prasek - Autoryzowany Dealer...


Impute values to remaining columns

In [503]:
imp = SimpleImputer(strategy="most_frequent")
df_imp = imp.fit_transform(df)
type(df_imp)

numpy.ndarray

In [504]:
df_filled = pd.DataFrame(df_imp, columns=df.columns)


In [505]:
df2 = pd.DataFrame()
for col in df_filled.columns:
    df2[col] = pd.to_numeric(df_filled[col], errors='ignore')


Split 'equipment' column into separate columns (filled with OHE)

In [506]:
def unique(list1):
    x = np.array(list1)
    return list(np.unique(x))

In [507]:
eq_list = ','.join(df['equipment'].to_list()).split(',')
eq_list = [x.strip() for x in eq_list]
eq_list = unique(eq_list)[1:]

In [508]:
df2_base = df2.copy()

In [509]:
def check_contain(col, str2):
    if str2 in col.split(','):
        return 1
    else:
        return 0

In [510]:
for item in eq_list:
    # df2[item]=0
    df2[item] = df2['equipment'].apply(lambda x: check_contain(x, item))

C:\Users\micha\AppData\Local\Temp\ipykernel_26972\2463573855.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df2[item] = df2['equipment'].apply(lambda x: check_contain(x, item))
C:\Users\micha\AppData\Local\Temp\ipykernel_26972\2463573855.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df2[item] = df2['equipment'].apply(lambda x: check_contain(x, item))
C:\Users\micha\AppData\Local\Temp\ipykernel_26972\2463573855.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.inser

Drop unnecessary cols

In [601]:
df_train = df2.copy().drop(['equipment', 'description'], axis=1)

Analyse categorical columns for number of uniqe values

In [602]:
dtypes_dict = df_train.dtypes.to_dict()

bool_cols = []
ohe_cols = []
drop_cols = []

for k in dtypes_dict:
    if dtypes_dict[k] == 'object':
        if df_train[k].nunique() == 2:
            print(df_train[k])
            bool_cols.append(k)
        elif df_train[k].nunique() < 2:
            drop_cols.append(k)
        else:
            ohe_cols.append(k)

0                   Firmy
1                   Firmy
2         Osoby prywatnej
3         Osoby prywatnej
4         Osoby prywatnej
               ...       
130799              Firmy
130800              Firmy
130801              Firmy
130802              Firmy
130803              Firmy
Name: Oferta od, Length: 130804, dtype: object
0             Manualna
1             Manualna
2             Manualna
3             Manualna
4             Manualna
              ...     
130799        Manualna
130800        Manualna
130801    Automatyczna
130802    Automatyczna
130803        Manualna
Name: Skrzynia biegów, Length: 130804, dtype: object
0         Używane
1         Używane
2         Używane
3         Używane
4         Używane
           ...   
130799    Używane
130800       Nowe
130801       Nowe
130802       Nowe
130803       Nowe
Name: Stan, Length: 130804, dtype: object


In [603]:
drop_cols

['Kategoria']

In [604]:
ohe_cols

['Marka pojazdu',
 'Model pojazdu',
 'Rodzaj paliwa',
 'Napęd',
 'Typ nadwozia',
 'Kolor']

In [605]:
bool_cols

['Oferta od', 'Skrzynia biegów', 'Stan']

In [606]:
df_encode = df_train.copy()

OHE categorical columns with more than two unique values

(temporarily does OHE for bool cols as well)

In [607]:
df_encode = pd.get_dummies(df_encode[ohe_cols + bool_cols])
df_encode

,Marka pojazdu_Abarth,Marka pojazdu_Acura,Marka pojazdu_Aixam,Marka pojazdu_Alfa Romeo,Marka pojazdu_Alpine,Marka pojazdu_Aston Martin,Marka pojazdu_Audi,Marka pojazdu_Austin,Marka pojazdu_BMW,Marka pojazdu_BMW-ALPINA,...,Kolor_Szary,Kolor_Zielony,Kolor_Złoty,Kolor_Żółty,Oferta od_Firmy,Oferta od_Osoby prywatnej,Skrzynia biegów_Automatyczna,Skrzynia biegów_Manualna,Stan_Nowe,Stan_Używane
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,1
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,1
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130799,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1
130800,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,1,1,0
130801,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,1,0
130802,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,1,0,1,0


In [608]:
df_train = df_train.drop(drop_cols + ohe_cols + bool_cols, axis=1)
df_train = pd.merge(left = df_train, right = df_encode, left_index=True, right_index=True)
df_train

,price,Rok produkcji,Przebieg,Pojemność skokowa,Moc,Liczba drzwi,Liczba miejsc,ABS,Aktywne rozpoznawanie znaków ograniczenia prędkości,Aktywny asystent hamowania awaryjnego,...,Kolor_Szary,Kolor_Zielony,Kolor_Złoty,Kolor_Żółty,Oferta od_Firmy,Oferta od_Osoby prywatnej,Skrzynia biegów_Automatyczna,Skrzynia biegów_Manualna,Stan_Nowe,Stan_Używane
0,39900.0,2009,158000.0,1368.0,215.0,3.0,5.0,1,0,0,...,0,0,0,0,1,0,0,1,0,1
1,72900.0,2019,66500.0,1368.0,145.0,3.0,4.0,1,0,0,...,0,0,0,0,1,0,0,1,0,1
2,39900.0,2012,193631.0,1400.0,135.0,3.0,4.0,1,0,0,...,0,0,0,0,0,1,0,1,0,1
3,43900.0,2015,139000.0,1368.0,160.0,3.0,4.0,0,0,0,...,0,0,0,0,0,1,0,1,0,1
4,36500.0,2013,103000.0,1368.0,160.0,3.0,4.0,1,0,0,...,0,0,0,0,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130799,124900.0,2021,14000.0,1499.0,130.0,5.0,5.0,1,1,0,...,0,0,0,0,1,0,0,1,0,1
130800,129900.0,2021,1.0,1199.0,130.0,5.0,5.0,1,0,0,...,1,0,0,0,1,0,0,1,1,0
130801,138300.0,2021,1.0,1199.0,130.0,5.0,5.0,1,0,0,...,0,0,0,0,1,0,1,0,1,0
130802,179000.0,2022,2.0,1199.0,130.0,5.0,5.0,1,0,0,...,1,0,0,0,1,0,1,0,1,0


In [609]:
for col in df_train.columns:
    print( col, df_train[col].max())

price 176157068.0
Rok produkcji 2023
Przebieg 2780000.0
Pojemność skokowa 9000.0
Moc 1500.0
Liczba drzwi 10.0
Liczba miejsc 9.0
ABS 1
Aktywne rozpoznawanie znaków ograniczenia prędkości 1
Aktywny asystent hamowania awaryjnego 1
Aktywny asystent zmiany pasa ruchu 1
Aktywny system monitorowania kondycji kierowcy 1
Alarm ruchu poprzecznego z tyłu pojazdu 1
Android Auto 1
Apple CarPlay 1
Asysten pasa ruchu 1
Asystent (czujnik) martwego pola 1
Asystent hamowania - Brake Assist 1
Asystent hamowania awaryjnego w mieście 1
Asystent jazdy w korku 1
Asystent pokonywania zakrętów 1
Asystent zapobiegania kolizjom na skrzyzowaniu 1
Asystent świateł drogowych 1
Automatyczna kontrola ogrzewania 1
Automatyczna kontrola zjazdu ze stoku 1
Autonomiczny system kierowania 1
Blokada mechanizmu różnicowego 1
Boczna poduszka powietrzna kierowcy 1
Boczne poduszki powietrzne - przód 1
Boczne poduszki powietrzne - tył 1
Cyfrowy kluczyk 1
Czujnik deszczu 1
Czujnik zmierzchu 1
Dach materiałowy 1
Dach otwierany ele

In [610]:
for col in df_train.columns:
    if df_train[col].max() > 10:
        # print(col)
        try:
            q_low = df_train[col].quantile(0.005)
            q_hi  = df_train[col].quantile(0.995)
            df_train = df_train[(df_train[col] < q_hi) & (df_train[col] > q_low)]
        except:
            # print('err - '+col)
            continue
df_train

price
Rok produkcji
Przebieg
Pojemność skokowa
Moc


,price,Rok produkcji,Przebieg,Pojemność skokowa,Moc,Liczba drzwi,Liczba miejsc,ABS,Aktywne rozpoznawanie znaków ograniczenia prędkości,Aktywny asystent hamowania awaryjnego,...,Kolor_Szary,Kolor_Zielony,Kolor_Złoty,Kolor_Żółty,Oferta od_Firmy,Oferta od_Osoby prywatnej,Skrzynia biegów_Automatyczna,Skrzynia biegów_Manualna,Stan_Nowe,Stan_Używane
0,39900.0,2009,158000.0,1368.0,215.0,3.0,5.0,1,0,0,...,0,0,0,0,1,0,0,1,0,1
1,72900.0,2019,66500.0,1368.0,145.0,3.0,4.0,1,0,0,...,0,0,0,0,1,0,0,1,0,1
2,39900.0,2012,193631.0,1400.0,135.0,3.0,4.0,1,0,0,...,0,0,0,0,0,1,0,1,0,1
3,43900.0,2015,139000.0,1368.0,160.0,3.0,4.0,0,0,0,...,0,0,0,0,0,1,0,1,0,1
4,36500.0,2013,103000.0,1368.0,160.0,3.0,4.0,1,0,0,...,0,0,0,0,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130794,55800.0,2016,93000.0,1199.0,130.0,5.0,5.0,1,0,0,...,0,0,0,0,1,0,0,1,0,1
130795,85000.0,2018,44000.0,1199.0,130.0,5.0,5.0,1,0,0,...,0,0,0,0,0,1,0,1,0,1
130796,39999.0,2017,53000.0,1199.0,130.0,5.0,5.0,1,1,0,...,0,1,0,0,0,1,0,1,0,1
130797,32800.0,2013,133800.0,1560.0,112.0,5.0,5.0,1,0,0,...,0,0,0,0,1,0,0,1,0,1


In [611]:
df_train.to_pickle('data/otomoto_data.pickle')